# Mutton Model

Model Network - CNN

# Data

In [45]:
import numpy as np
import pandas as pd
from os import path
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [42]:
# importing data
root_path_ = './gdrive/My Drive/2020/Mutton/github'
training_data_path = root_path + '/data/_Train.csv'
## 너 한테 맞는 루트로 설정해야함

X_train = pd.read_csv(path_).values
print(X_train.shape)


# Graph Plotting
import matplotlib.pyplot as plt

plt.figure(figsize=(10,5))

plt.ylabel('Amplitude [uV]')
plt.xlabel('Time [5ms]')
rData = X_train[:,0]
rDataNum = rData.shape[0]
plt.plot(range(rDataNum), rData)
rData = X_train[:,1]
rDataNum = rData.shape[0]
plt.plot(range(rDataNum), rData)

(43364, 3)


Preprocessing (Windowing, Filtering)

In [46]:
# Window Streaming
seq_len = 512

def window_data(seq_len,data) :
    res = [] # n * (128 X 2)
    num_ch = 2
    for i in range(data.shape[0] - seq_len + 1):
        tmp = []
        tmp.append(data[i:i+seq_len, 0:2])
        
        class_label = int(data[i,2])
        if(class_label == 0):
            tmp.append(0)
        elif(class_label == 1):
            tmp.append(1)
        elif(class_label == 2):
            tmp.append(2)
        elif(class_label == 3):
            tmp.append(3)
        
        res.append(tmp)
    return np.asarray(res)

training_data = window_data(seq_len, X_train)

input_train = training_data[:, 0]
label_train = training_data[:, 1]


# Filtering
from scipy import signal

def sine_generator(fs, sinefreq, duration):
    T = duration
    nsamples = fs * T
    w = 2. * np.pi * sinefreq
    t_sine = np.linspace(0, T, nsamples, endpoint=False)
    y_sine = np.sin(w * t_sine)
    result = pd.DataFrame({ 
        'data' : y_sine} ,index=t_sine)
    return result

def butter_highpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = signal.butter(order, normal_cutoff, btype='high', analog=False)
    return b, a

def butter_highpass_filter(data, cutoff, fs, order=5):
    b, a = butter_highpass(cutoff, fs, order=order)
    y = signal.filtfilt(b, a, data)
    return y

train_input = []
for data in input_train :
    train_input.append(data)
train_input = np.asarray(train_input)
print(train_input.shape)

label_input = []
for data in label_train :
    label_input.append(data)
label_input = np.asarray(label_input)
print(label_input.shape)

fps = 200

fData0 = []
fData1 = []
for data in train_input:
    data[:,0] = butter_highpass_filter(data[:,0],50,fps)
    data[:,1] = butter_highpass_filter(data[:,1],50,fps)

    fData0.append(data[0,0])
    fData1.append(data[0,1])
fData = []
fData.append(fData0)
fData.append(fData1)
fData = np.asarray(fData)

print(train_input.shape)
print(label_input.shape)
print(fData.shape)

(42853, 512, 2)
(42853,)
(42853, 512, 2)
(42853,)
(2, 42853)


To Feed into the Network

In [101]:
# Valid Set Split
from sklearn.model_selection import train_test_split

X_tr, X_vld, lab_tr, lab_vld = train_test_split(
    train_input, label_input,
    stratify = label_input, random_state = 123)

X_tr = X_tr.reshape(len(X_tr), -1, 1)
X_vld = X_vld.reshape(len(X_vld), -1, 1)

print(X_tr.shape)
print(X_vld.shape)
print(lab_tr.shape)
print(lab_vld.shape)

# Building tf.DataSet
dbs = tf.data.Dataset.from_tensor_slices((X_tr,lab_tr))
dbs = dbs.batch(128).repeat(3)

dbt = tf.data.Dataset.from_tensor_slices((X_vld,lab_vld))
dbt = dbt.batch(1).repeat(3)

print(dbs)
print(dbt)

(32139, 1024, 1)
(10714, 1024, 1)
(32139,)
(10714,)
<RepeatDataset shapes: ((None, 1024, 1), (None,)), types: (tf.float64, tf.int64)>
<RepeatDataset shapes: ((None, 1024, 1), (None,)), types: (tf.float64, tf.int64)>


# Model (CNN)

In [102]:
import  tensorflow as tf
from    tensorflow import keras
from    tensorflow.keras import datasets, layers, optimizers, models

In [106]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv1D(input_shape=((1024,1)), kernel_size=128, filters=16, padding = 'same'),
  tf.keras.layers.AveragePooling1D(pool_size=2, strides=2),
  tf.keras.layers.Conv1D(input_shape=((512,1)), kernel_size=128, filters=32, padding = 'same'),
  tf.keras.layers.AveragePooling1D(pool_size=2, strides=2),
  tf.keras.layers.Conv1D(input_shape=((256,1)), kernel_size=64, filters=64, padding = 'same'),
  tf.keras.layers.AveragePooling1D(pool_size=2, strides=2),
  tf.keras.layers.Conv1D(input_shape=((128,1)), kernel_size=64, filters=128, padding = 'same'),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(32, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(4, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential_30"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_62 (Conv1D)           (None, 1024, 16)          2064      
_________________________________________________________________
average_pooling1d_27 (Averag (None, 512, 16)           0         
_________________________________________________________________
conv1d_63 (Conv1D)           (None, 512, 32)           65568     
_________________________________________________________________
average_pooling1d_28 (Averag (None, 256, 32)           0         
_________________________________________________________________
conv1d_64 (Conv1D)           (None, 256, 64)           131136    
_________________________________________________________________
average_pooling1d_29 (Averag (None, 128, 64)           0         
_________________________________________________________________
conv1d_65 (Conv1D)           (None, 128, 128)        

# Training & Testing

In [ ]:
for _ in range(5): model.fit(dbs)
model.evaluate(dbt)

118/756 [===>..........................] - ETA: 36:54 - loss: 1.1209 - accuracy: 0.5393